In [ ]:
!pip install split-folders
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import tensorflow 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow import keras

from tensorflow.keras.applications.vgg16 import VGG16 # VGG16
from tensorflow.keras.applications.vgg19 import VGG19 # VGG19
from tensorflow.keras.applications.resnet50 import ResNet50 # ResNet50
from tensorflow.keras.applications import ResNet101 # ResNet 101
from tensorflow.keras.applications.xception import Xception # Xception
from tensorflow.keras.applications.mobilenet import MobileNet # MobileNet
from tensorflow.keras.applications.densenet import DenseNet169 # DenseNet169
from tensorflow.keras.applications.densenet import DenseNet121 # DenseNet121
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 # MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3 # InceptionV3

from tqdm import tqdm
from random import shuffle  
from zipfile import ZipFile
from PIL import Image
import cv2
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import Model
from tensorflow.keras import callbacks
import splitfolders


In [ ]:
"""
classes = {"AbdomenCT" :0 ,"BreastMRI" : 1,"CXR" : 2,"ChestCT": 3,"Hand": 4,"HeadCT": 5}
img_size = (64,64)
abdomen_DIR='/kaggle/input/medical-mnist/AbdomenCT'
breastmri_DIR='/kaggle/input/medical-mnist/BreastMRI'
chestct_DIR='/kaggle/input/medical-mnist/ChestCT'
cxr_DIR='/kaggle/input/medical-mnist/CXR'
hand_DIR='/kaggle/input/medical-mnist/Hand'
headct = '/kaggle/input/medical-mnist/HeadCT'
"""

In [ ]:
splitfolders.ratio("../input/medical-mnist",ratio = (0.8,0.1,0.1))

In [ ]:
generator = ImageDataGenerator(rescale=1/255, validation_split=0)

train_data =  generator.flow_from_directory(batch_size=64,
                            directory='./output/train',
                            shuffle=True,
                            target_size=(64,64), 
                            class_mode='categorical')

val_data =  generator.flow_from_directory(batch_size=64,
                            directory='./output/val',
                            shuffle=True,
                            target_size=(64,64),
                            class_mode='categorical')

test_data =  generator.flow_from_directory(batch_size=64,
                            directory='./output/test',
                            shuffle=False,
                            target_size=(64,64), 
                            class_mode='categorical')


In [ ]:
vgg = VGG19(input_shape=(64,64,3), weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False
x = layers.Flatten()(vgg.output)

prediction = layers.Dense(6, activation='softmax')(x)

modelvgg = Model(inputs=vgg.input, outputs=prediction)
modelvgg.summary()

In [ ]:
modelvgg.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=2,
                                            restore_best_weights=True)
modelvgg.fit(train_data, epochs=5, validation_data=val_data, callbacks=callback)

In [ ]:
loss , auc, acc = modelvgg.evaluate(test_data)


In [ ]:
train_images = train_images.reshape((42000, 28 * 28))
train_images = train_images.astype("float32") / 255

test_images = test_images.reshape((28000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [ ]:
model = models.Sequential([
    keras.layers.Flatten(input_shape = [64,64,3]), 
    layers.Dense(512,activation = "relu"),
    layers.Dense(512,activation = "relu"),
    layers.Dense(6,activation = "softmax")
])

model.compile(optimizer = "adam",
                loss = "categorical_crossentropy",
                metrics = "accuracy")

model.fit(train_data,epochs = 5)



In [ ]:
loss,acc = model.evaluate(test_data)